# Vienna BUILDINGS

the row vector datasets :



1. Gebäudeinformationen Allgemein  https://www.data.gv.at/katalog/dataset/7a8aae59-71a4-4500-b38b-bdf15c7f627f
OBJECTID ACD (=Adresscode)
STRCD (=Straßencode) STRNAML (=Straßenname)
VONN (=Ordnungsnummer von - Numerisch) 
VONA (=Ordnungsnummer von - Alphanumerisch) BISN (=Ordnungsnummer bis - Numerisch) BISA (=Ordnungsnummer bis - Alphanumerisch) STGN (=Stiegennummer von - Numerisch) STGA (=Stiegennummer von - Alphanumerisch) BEZ (=Bezirk) HA_NAME (=Anlagenname) BAUJAHR (= Baujahr des Gebäudes)
ARCHITEKT (= Der Architekt)
SHAPE (=Geometrie)
IDENT_1 (=Identadresse Nummer 1)
IDENT_2 (=Identadresse Nummer 2)
IDENT_3 (=Identadresse Nummer 3)
CONSCR_NR (=Conscriptionsnummer) 
PR_MA19 (=präferierte Quelle MA19) 
GESCH_ANZ (=Geschoßanzahl) 
BASIS_INV (=Inventarisierungstand) 
L_NUTZUNG (=Gebäudenutzung) 
L_BAUTYP (=Bautyp) 
NS (=Geschoße) wobei: D=Dachgeschoß, K=Keller, S= Souterrain ORIG_PARZ ( -1 = ensembletypische Parzellenstruktur) ORIG_FASS ( -1 = erhaltene Fassade) ORIG_KERN ( -1 = erhaltene Struktur des Kernes) ORIG_FENSTER ( -1 = erhaltene Fenster) ORIG_DACH ( -1 = erhaltenes Dach)

drive:
"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\buildings\baukoerpermodle_lod14\GEBAEUDEINFOOGDPoint.shp"


2. FMZK - Gebäude https://www.data.gv.at/katalog/dataset/76c2e577-268f-4a93-bccd-7d5b43b14efd
AYER: Textliche Beschreibung für Attribut F_KLASSE BEZUG: Adresscode für Gebäude 
H_KLASSE: Höhenklassen für Gebäudehöhenplan 
KLASSE_SUB: Definition der Bodennutzung der unteren Fläche, wenn Flächen übereinander liegen; Wertebereich wie bei Attribut F_KLASSE 
O_KOTE: absolute Gebäudehöhe der Dachtraufe 
U_KOTE: absolute Überbauungshöhe unten
HOEHE_DGM: absolute Geländehöhe
T_KOTE: tiefster Punkt des Geländes auf den Kanten der Gebäudeteilfläche 
FMZK_ID: Eindeutiger Flächenbezug der FMZK 
BW_GEB_ID: Adresscode für zusammengehörige Gebäudeflächen 
BW_BRK_ID: Adresscode für zusammengehörige Brückenflächen 
BW_SON_ID: Adresscode für zusammengehörige sonstige Bauwerksflächen

drive:
"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\buildings\baukoerpermodle_lod14\BUILDINGS_model.shp"

-----------------------

1) download files
2) start the scripts:

were already mergen in QGIS and stored as a 10m raster file in 31256 projection:
"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Mehrzweckkarte\FMZK_WIEN_GP\fmzk_gewaesser_poly_shp31256.shp"


## Part 1 - using UNION of vector datasets: 




In [27]:
## reading files....

# geometry:
import geopandas as gpd
import os
   
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

import numpy as np


##input:
in_vector_datasest = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\buildings\baukoerpermodle_lod14\BUILDINGS_model.shp"
## reference poly: 100x100m:
reference_polygon = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp"


name ='buildings'
## processing:
union_shp = os.path.splitext(in_vector_datasest)[0]+"_union.shp"
#print (union_shp)


## out:
out_100m_vector =         r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_"+name+"_by_grid100m.shp"
out_100m_vector_indicator=r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_"+name+"_areas_by_grid100m_indicator.shp"


print ("datasets found")


# 1. transformation of land use vector dataset to the 100m Vienna GRID:
# 1 Load GRID
shp_100m_GRID = gpd.read_file(reference_polygon)
# 2 LOAD "land use"
shp = gpd.read_file(in_vector_datasest)

# Perform spatial union
union_result = gpd.overlay(shp_100m_GRID, shp, how='union')
union_result.to_file(union_shp)
# Save the result to a new shapefile


print ("done")










datasets found


C:\Users\Admin\AppData\Local\Temp\ipykernel_17132\1180925346.py:41: UserWarning: `keep_geom_type=True` in overlay resulted in 741 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  union_result = gpd.overlay(shp_100m_GRID, shp, how='union')


done


## Part 1.1 - merge to 100m vector cells and calculation of indicators (building foodprint area & volumn)

In [43]:
## calc area by cell
union_result['area'] = union_result.geometry.area
union_result['area_ha'] = union_result['area'].astype(float) / 10000




 ##1 building foodprint: ------------------------------------
#grouped_stats = union_result.groupby(['GRID100mID','LAYER'])['area'].sum()
grouped_stats = union_result[union_result['H_KLASSE'] > 0].groupby(['GRID100mID'])['area_ha'].sum()   # sum only area where buildings can be found  - building foodprint
grouped_stats['buildings'] ='buildings'
#grouped_stats
grouped_stats = grouped_stats.reset_index()
## joint group by table to orginal 100m GRIDcells:
merged_gdf = shp_100m_GRID.merge(grouped_stats, on='GRID100mID', how='left')
merged_gdf['area_ha'] = merged_gdf['area_ha'].astype(float) 

# normalize the data
merged_gdf['area_ha'] = merged_gdf['area_ha'].fillna(0)   # remove NULL values
merged_gdf['area_ha_i']= (merged_gdf['area_ha']-min(merged_gdf['area_ha']))/(max(merged_gdf['area_ha'])-min(merged_gdf['area_ha']))*100  #normalization


 ##2 building hight:   [O_KOTE]------------------------------------
grouped_stats_2 = union_result[union_result['H_KLASSE'] > 0].groupby(['GRID100mID'])['O_KOTE'].mean()   # sum only area where buildings can be found  - building foodprint
grouped_stats_2['build_hight'] ='avg_building_hight'


merged_gdf = merged_gdf.merge(grouped_stats_2, on='GRID100mID', how='left')
merged_gdf['O_KOTE'] = merged_gdf['O_KOTE'].astype(float) 



 ##3 building volumn:   -----------------------------------
merged_gdf['build_vol'] = merged_gdf['O_KOTE'] * merged_gdf['area_ha']   ## calculation ov volumne
merged_gdf['build_vol'] = merged_gdf['build_vol'].fillna(0)   # remove NULL values


# normalize the data
merged_gdf['build_v_i']= (merged_gdf['build_vol']-min(merged_gdf['build_vol']))/(max(merged_gdf['build_vol'])-min(merged_gdf['build_vol']))*100  #normalization
#print("value of max and mean", merged_gdf['build_v_i'].max(), merged_gdf['build_v_i'].min()) 





############################################################################################################################################################################
## export group by data frame to vector dataset:
merged_gdf.to_file(out_100m_vector)
print ("vector union done")


######### 




print ("done")


vector union done
done


# reclassify data frame (all water pixeld to value 1)

## Part 1.2 Export vector data to raster datasets:

In [46]:
##

## raster to vector using vector attriubte (working perfectly):
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds


## export vector 100x100m cells to raster:

polygons = gpd.read_file(out_100m_vector)    ## reading the union vector dataset




# Define the attribute to use for raster values
attribute_to_rasterize_1_area = 'area_ha_i'
attribute_to_rasterize_2_vol   = 'build_v_i'

output_raster_1_area =         r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\r08_Wien_"+attribute_to_rasterize_1_area+"_grid100m.tif"    # output area indicator
output_raster_2_vol =          r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\r09_Wien_"+attribute_to_rasterize_2_vol+"_grid100m.tif"    # output area indicator




## 1 Area indicator--------------------------------------------------------------------------------------------------
# Define raster properties
#xmin, ymin, xmax, ymax = polygons.total_bounds

attribute_to_rasterize = attribute_to_rasterize_1_area
output_raster = output_raster_1_area 

xmin=	-11200
ymin=	331000
xmax=	18800
ymax=	354000
projection ='epsg:31256'
pixel_size = 100  # Adjust as needed

width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

# Create an empty array to hold rasterized values
raster_array = rasterize(
    [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=False,  # or True
    dtype=rasterio.float32
)


# Write the raster to a GeoTIFF file
with rasterio.open(output_raster, "w", driver="GTiff", 
                   height=raster_array.shape[0], 
                   width=raster_array.shape[1], 
                   count=1, 
                   dtype=raster_array.dtype, 
                   crs=polygons.crs, 
                   transform=transform
                   ) as dst:
    dst.write(raster_array, 1)

print ("-------------------------------")
print ("raster 1 ready")
print (output_raster)

## 2 Vol indicator--------------------------------------------------------------------------------------------------
# Define raster properties
#xmin, ymin, xmax, ymax = polygons.total_bounds

attribute_to_rasterize = attribute_to_rasterize_2_vol
output_raster          = output_raster_2_vol 


# Create an empty array to hold rasterized values
raster_array = rasterize(
    [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=False,  # or True
    dtype=rasterio.float32
)


# Write the raster to a GeoTIFF file
with rasterio.open(output_raster, "w", driver="GTiff", 
                   height=raster_array.shape[0], 
                   width=raster_array.shape[1], 
                   count=1, 
                   dtype=raster_array.dtype, 
                   crs=polygons.crs, 
                   transform=transform
                   ) as dst:
    dst.write(raster_array, 1)


print ("-------------------------------")
print ("raster 2 ready")
print (output_raster)

#--------------------------------------------------------------------------------------------------------------------------
print ("done")

-------------------------------
raster 1 ready
N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\r08_Wien_area_ha_i_grid100m.tif
-------------------------------
raster 2 ready
N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\r09_Wien_build_v_i_grid100m.tif
done


## 